# Authentication and initializing Spotipy API

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [5]:
CLIENT_ID = "9c5219359a95470a9f3502927e7751bd"
CLIENT_SECRET = "33491abd2b804276b6d9921e64e4d24f"

In [6]:

!pip install spotipy

In [7]:


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=CLIENT_ID,
                                                           client_secret=CLIENT_SECRET))


In [8]:
results = sp.search(q='Lady Gaga', limit=50)
results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Lady+Gaga&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
       'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
       'id': '1HY2Jd0NmPuamShAr6KMms',
       'name': 'Lady Gaga',
       'type': 'artist',
       'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/4VIvfOurcf0vuLRxLkGnIG'},
       'href': 'https://api.spotify.com/v1/artists/4VIvfOurcf0vuLRxLkGnIG',
       'id': '4VIvfOurcf0vuLRxLkGnIG',
       'name': 'Bradley Cooper',
       'type': 'artist',
       'uri': 'spotify:artist:4VIvfOurcf0vuLRxLkGnIG'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'D

In [9]:
dir(sp)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_append_device_id',
 '_auth',
 '_auth_headers',
 '_auth_manager',
 '_build_session',
 '_delete',
 '_get',
 '_get_id',
 '_get_uri',
 '_internal_call',
 '_is_uri',
 '_post',
 '_put',
 '_regex_base62',
 '_regex_spotify_uri',
 '_regex_spotify_url',
 '_search_multiple_markets',
 '_session',
 'add_to_queue',
 'album',
 'album_tracks',
 'albums',
 'artist',
 'artist_albums',
 'artist_related_artists',
 'artist_top_tracks',
 'artists',
 'audio_analysis',
 'audio_features',
 'auth_manager',
 'available_markets',
 'backoff_factor',
 'categories',
 'category',
 'category_playlists',
 'client_credentials

# GIT

## Getting songs from individual playlists

In [12]:
def get_playlist_tracks(username,playlist_id):
    
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= CLIENT_ID, client_secret=CLIENT_SECRET))
    
    results = sp.user_playlist_tracks(username,playlist_id)
    
    tracks = results["items"]
    
    while results["next"]:
        results = sp.next(results)
        tracks.extend(results["items"])
        
    return tracks

In [13]:
def get_values(playlist_list):


    song_names = []
    artist_names = []
    song_uri = []
    song_uri_total = []
    song_features = []
    

    counter = 0

    for item in playlist_list:
        
        try:
            
            counter = counter + 1 
    
            song_names.append(item['track']['name'])
    
            song_uri_total.append(item['track']['uri'])
    
            song_uri.append(item['track']['uri'])
        
    
            tempo_artists = []
            for artist in item['track']['artists']:
                tempo_artists.append(artist['name'])
    
            artist_names.append(tempo_artists)
    
            if counter == 100:
                song_features.append(sp.audio_features(song_uri))
                counter = 0
                song_uri = []
                
        except TypeError:
            
            continue
            
            
    song_features.append(sp.audio_features(song_uri))
    
    #print(artist_names)
    #artist_names = np.array(artist_names).flatten().tolist()
    
    song_id_total = [ uri[14:] for uri in song_uri_total ]
    
    song_features_total = [item for songs in song_features for item in songs]
    
    artist_names = [", ".join(artist) for artist in artist_names]
    
    my_dict = {'title': song_names, 'artist': artist_names, 'uri': song_uri_total,'features': song_features_total }  
    
    songs_df = pd.DataFrame(my_dict)
    
    songs_df = pd.concat([songs_df.drop(columns='features'),songs_df['features'].apply(pd.Series)],axis=1)
    
    return songs_df

In [14]:
def audio_features(x):
    
    song = sp.search(q = x, limit = 1) 
    uri = song["tracks"]["items"][0]["uri"]
    
    
    audiofeatures = sp.audio_features(uri)[0]
    

    song_df = pd.DataFrame(audiofeatures,index=[0])
    
    song_df = song_df.drop(columns=['key','type','uri','id','mode','track_href','analysis_url','duration_ms','time_signature'])
    
    
    
    return song_df

In [15]:
#Best Hits 2024
#https://open.spotify.com/playlist/5KJDMJe9EJ7QRz8FG2MIpI

play1 = get_playlist_tracks("spotify", "5KJDMJe9EJ7QRz8FG2MIpI")

In [16]:
df1 = get_values(play1)

df1.head()

,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,BIRDS OF A FEATHER,Billie Eilish,spotify:track:6dOtVTDdiauQNBQEDOtlAB,0.747,0.507,2,-10.171,1,0.0358,0.200,...,0.117,0.438,104.978,audio_features,6dOtVTDdiauQNBQEDOtlAB,spotify:track:6dOtVTDdiauQNBQEDOtlAB,https://api.spotify.com/v1/tracks/6dOtVTDdiauQ...,https://api.spotify.com/v1/audio-analysis/6dOt...,210373,4
1,Espresso,Sabrina Carpenter,spotify:track:2qSkIjg1o9h3YT9RAgYN75,0.701,0.760,0,-5.478,1,0.0285,0.107,...,0.185,0.690,103.969,audio_features,2qSkIjg1o9h3YT9RAgYN75,spotify:track:2qSkIjg1o9h3YT9RAgYN75,https://api.spotify.com/v1/tracks/2qSkIjg1o9h3...,https://api.spotify.com/v1/audio-analysis/2qSk...,175459,4
2,Die With A Smile,"Lady Gaga, Bruno Mars",spotify:track:2plbrEY59IikOBgBGLjaoe,0.521,0.592,6,-7.777,0,0.0304,0.308,...,0.122,0.535,157.969,audio_features,2plbrEY59IikOBgBGLjaoe,spotify:track:2plbrEY59IikOBgBGLjaoe,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,https://api.spotify.com/v1/audio-analysis/2plb...,251668,3
3,As It Was,Harry Styles,spotify:track:4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,0.342,...,0.311,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,Life,ConKi,spotify:track:5LOTtadWBsGDLMQMC0tl2z,0.607,0.817,0,-5.839,0,0.0485,0.130,...,0.560,0.338,112.052,audio_features,5LOTtadWBsGDLMQMC0tl2z,spotify:track:5LOTtadWBsGDLMQMC0tl2z,https://api.spotify.com/v1/tracks/5LOTtadWBsGD...,https://api.spotify.com/v1/audio-analysis/5LOT...,130000,4


In [17]:
df1.shape

(148, 21)

In [18]:
#https://open.spotify.com/playlist/1iUWApPD3AdyoeBBDb1ibl
#English Top Hits 2023 - 2024

play2 = get_playlist_tracks("spotify", "1iUWApPD3AdyoeBBDb1ibl")
df2 = get_values(play2)
print(df2.shape)
df2.head()

(106, 21)


,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,BIRDS OF A FEATHER,Billie Eilish,spotify:track:6dOtVTDdiauQNBQEDOtlAB,0.747,0.507,2,-10.171,1,0.0358,0.2000,...,0.117,0.438,104.978,audio_features,6dOtVTDdiauQNBQEDOtlAB,spotify:track:6dOtVTDdiauQNBQEDOtlAB,https://api.spotify.com/v1/tracks/6dOtVTDdiauQ...,https://api.spotify.com/v1/audio-analysis/6dOt...,210373,4
1,Please Please Please,Sabrina Carpenter,spotify:track:5N3hjp1WNayUPZrA8kJmJP,0.669,0.586,9,-6.073,1,0.0540,0.2740,...,0.104,0.579,107.071,audio_features,5N3hjp1WNayUPZrA8kJmJP,spotify:track:5N3hjp1WNayUPZrA8kJmJP,https://api.spotify.com/v1/tracks/5N3hjp1WNayU...,https://api.spotify.com/v1/audio-analysis/5N3h...,186365,4
2,Big Dawgs,"Hanumankind, Kalmi",spotify:track:0OA00aPt3BV10qeMIs3meW,0.464,0.745,4,-3.202,0,0.1610,0.0235,...,0.363,0.262,180.098,audio_features,0OA00aPt3BV10qeMIs3meW,spotify:track:0OA00aPt3BV10qeMIs3meW,https://api.spotify.com/v1/tracks/0OA00aPt3BV1...,https://api.spotify.com/v1/audio-analysis/0OA0...,190667,4
3,Espresso,Sabrina Carpenter,spotify:track:2qSkIjg1o9h3YT9RAgYN75,0.701,0.760,0,-5.478,1,0.0285,0.1070,...,0.185,0.690,103.969,audio_features,2qSkIjg1o9h3YT9RAgYN75,spotify:track:2qSkIjg1o9h3YT9RAgYN75,https://api.spotify.com/v1/tracks/2qSkIjg1o9h3...,https://api.spotify.com/v1/audio-analysis/2qSk...,175459,4
4,Guess featuring billie eilish,"Charli xcx, Billie Eilish",spotify:track:7qbJ4Dh8xuYUFdAWL8SFPK,0.780,0.713,0,-4.493,1,0.0888,0.0167,...,0.144,0.615,130.034,audio_features,7qbJ4Dh8xuYUFdAWL8SFPK,spotify:track:7qbJ4Dh8xuYUFdAWL8SFPK,https://api.spotify.com/v1/tracks/7qbJ4Dh8xuYU...,https://api.spotify.com/v1/audio-analysis/7qbJ...,143331,4


In [19]:
#https://open.spotify.com/playlist/7vo8wCFNJS0Bn5yiN491KP
#Best of 2024

play3 = get_playlist_tracks("spotify", "7vo8wCFNJS0Bn5yiN491KP")
df3 = get_values(play3)
print(df3.shape)
df3.head()

(952, 21)


,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Die With A Smile,"Lady Gaga, Bruno Mars",spotify:track:2plbrEY59IikOBgBGLjaoe,0.521,0.592,6,-7.777,0,0.0304,0.308000,...,0.1220,0.535,157.969,audio_features,2plbrEY59IikOBgBGLjaoe,spotify:track:2plbrEY59IikOBgBGLjaoe,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,https://api.spotify.com/v1/audio-analysis/2plb...,251668,3
1,The Emptiness Machine,Linkin Park,spotify:track:2PnlsTsOTLE5jnBnNe2K0A,0.466,0.872,7,-3.344,1,0.0336,0.015600,...,0.1210,0.806,184.115,audio_features,2PnlsTsOTLE5jnBnNe2K0A,spotify:track:2PnlsTsOTLE5jnBnNe2K0A,https://api.spotify.com/v1/tracks/2PnlsTsOTLE5...,https://api.spotify.com/v1/audio-analysis/2Pnl...,190428,4
2,i like the way you kiss me,Artemas,spotify:track:2GxrNKugF82CnoRFbQfzPf,0.599,0.946,11,-4.263,1,0.0447,0.000938,...,0.0826,0.747,151.647,audio_features,2GxrNKugF82CnoRFbQfzPf,spotify:track:2GxrNKugF82CnoRFbQfzPf,https://api.spotify.com/v1/tracks/2GxrNKugF82C...,https://api.spotify.com/v1/audio-analysis/2Gxr...,142515,4
3,APT.,"ROSÉ, Bruno Mars",spotify:track:5vNRhkKd0yEAg8suGBpjeY,0.777,0.783,0,-4.477,0,0.2600,0.028300,...,0.3550,0.939,149.027,audio_features,5vNRhkKd0yEAg8suGBpjeY,spotify:track:5vNRhkKd0yEAg8suGBpjeY,https://api.spotify.com/v1/tracks/5vNRhkKd0yEA...,https://api.spotify.com/v1/audio-analysis/5vNR...,169917,4
4,It Girl (feat. Sampagne),"Shirin David, Sampagne",spotify:track:2byoNXF9Y6qhIGBq33ATEm,0.919,0.623,2,-5.312,0,0.1010,0.050800,...,0.0587,0.756,127.032,audio_features,2byoNXF9Y6qhIGBq33ATEm,spotify:track:2byoNXF9Y6qhIGBq33ATEm,https://api.spotify.com/v1/tracks/2byoNXF9Y6qh...,https://api.spotify.com/v1/audio-analysis/2byo...,143622,4


## Concatenating mupltiple latest dataframes into single dataframe

In [21]:
#Concatenating mupltiple latest dataframes into single dataframe

frames = [df1,df2,df3]

song_database= pd.concat(frames)
print(song_database.shape)
song_database

(1206, 21)


,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,BIRDS OF A FEATHER,Billie Eilish,spotify:track:6dOtVTDdiauQNBQEDOtlAB,0.747,0.507,2,-10.171,1,0.0358,0.20000,...,0.1170,0.438,104.978,audio_features,6dOtVTDdiauQNBQEDOtlAB,spotify:track:6dOtVTDdiauQNBQEDOtlAB,https://api.spotify.com/v1/tracks/6dOtVTDdiauQ...,https://api.spotify.com/v1/audio-analysis/6dOt...,210373,4
1,Espresso,Sabrina Carpenter,spotify:track:2qSkIjg1o9h3YT9RAgYN75,0.701,0.760,0,-5.478,1,0.0285,0.10700,...,0.1850,0.690,103.969,audio_features,2qSkIjg1o9h3YT9RAgYN75,spotify:track:2qSkIjg1o9h3YT9RAgYN75,https://api.spotify.com/v1/tracks/2qSkIjg1o9h3...,https://api.spotify.com/v1/audio-analysis/2qSk...,175459,4
2,Die With A Smile,"Lady Gaga, Bruno Mars",spotify:track:2plbrEY59IikOBgBGLjaoe,0.521,0.592,6,-7.777,0,0.0304,0.30800,...,0.1220,0.535,157.969,audio_features,2plbrEY59IikOBgBGLjaoe,spotify:track:2plbrEY59IikOBgBGLjaoe,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,https://api.spotify.com/v1/audio-analysis/2plb...,251668,3
3,As It Was,Harry Styles,spotify:track:4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,0.34200,...,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4
4,Life,ConKi,spotify:track:5LOTtadWBsGDLMQMC0tl2z,0.607,0.817,0,-5.839,0,0.0485,0.13000,...,0.5600,0.338,112.052,audio_features,5LOTtadWBsGDLMQMC0tl2z,spotify:track:5LOTtadWBsGDLMQMC0tl2z,https://api.spotify.com/v1/tracks/5LOTtadWBsGD...,https://api.spotify.com/v1/audio-analysis/5LOT...,130000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,Nice To Meet You,Myles Smith,spotify:track:726SEjbF3ToEbYbn43gnxl,0.762,0.783,5,-5.206,0,0.0387,0.32900,...,0.0536,0.840,115.091,audio_features,726SEjbF3ToEbYbn43gnxl,spotify:track:726SEjbF3ToEbYbn43gnxl,https://api.spotify.com/v1/tracks/726SEjbF3ToE...,https://api.spotify.com/v1/audio-analysis/726S...,176000,4
948,Love Is The Only Thing,Lost Frequencies,spotify:track:4r6c2ClnuYLwNv4lEM7XlW,0.608,0.688,5,-8.597,0,0.0499,0.05540,...,0.0800,0.214,127.938,audio_features,4r6c2ClnuYLwNv4lEM7XlW,spotify:track:4r6c2ClnuYLwNv4lEM7XlW,https://api.spotify.com/v1/tracks/4r6c2ClnuYLw...,https://api.spotify.com/v1/audio-analysis/4r6c...,171094,4
949,Over You (feat. Justin Jesso),"Ofenbach, Justin Jesso",spotify:track:69GEK9C8nKwiGLMv2NbpiG,0.746,0.753,9,-4.667,0,0.2340,0.00435,...,0.0600,0.848,115.865,audio_features,69GEK9C8nKwiGLMv2NbpiG,spotify:track:69GEK9C8nKwiGLMv2NbpiG,https://api.spotify.com/v1/tracks/69GEK9C8nKwi...,https://api.spotify.com/v1/audio-analysis/69GE...,173922,4
950,Drum (feat. Lena Sue),"Alle Farben, Parov Stelar, Lena Sue",spotify:track:0PaxddTF3iYgbPBHitXX6V,0.868,0.888,9,-4.443,1,0.0458,0.06670,...,0.0454,0.969,124.017,audio_features,0PaxddTF3iYgbPBHitXX6V,spotify:track:0PaxddTF3iYgbPBHitXX6V,https://api.spotify.com/v1/tracks/0PaxddTF3iYg...,https://api.spotify.com/v1/audio-analysis/0Pax...,169718,4


In [22]:
song_database.columns

Index(['title', 'artist', 'uri', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature'],
      dtype='object')

In [23]:
#Dropping all the non useful columns

song_database = song_database.drop(columns=['type','uri','id','mode','track_href','analysis_url','duration_ms','time_signature'])

In [24]:
song_database.columns

Index(['title', 'artist', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo'],
      dtype='object')

## Dropping duplicates and cleaning

In [26]:
#Checking for missing values
song_database.isna().sum()

title               0
artist              0
danceability        0
energy              0
key                 0
loudness            0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
dtype: int64

No missing values

In [28]:
song_database.shape

(1206, 12)

In [29]:
#Dropping duplicates

song_database = song_database.drop_duplicates()

In [30]:
song_database.shape

(1143, 12)

In [31]:
# Save to CSV
#song_database.to_csv('spotify_song.csv', index=False)

# Global Hits

In [58]:
play_global = get_playlist_tracks("spotify", "37i9dQZEVXbMDoHDwVN2tF")
df_global = get_values(play_global)
print(df_global.shape)
df_global.tail()

(50, 21)


,title,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
45,"Sticky (feat. GloRilla, Sexyy Red & Lil Wayne)","Tyler, The Creator, GloRilla, Sexyy Red, Lil W...",spotify:track:3tFed7YsjGnIfxeLEQwx3R,0.505,0.812,1,-3.986,1,0.4610,0.28500,...,0.4440,0.664,80.090,audio_features,3tFed7YsjGnIfxeLEQwx3R,spotify:track:3tFed7YsjGnIfxeLEQwx3R,https://api.spotify.com/v1/tracks/3tFed7YsjGnI...,https://api.spotify.com/v1/audio-analysis/3tFe...,255916,4
46,Presidente,"Gabito Ballesteros, Natanael Cano, Luis R Conr...",spotify:track:4OLT65TRsOx3Iv1TlCcQVb,0.732,0.827,11,-4.152,1,0.0585,0.12300,...,0.1170,0.584,105.145,audio_features,4OLT65TRsOx3Iv1TlCcQVb,spotify:track:4OLT65TRsOx3Iv1TlCcQVb,https://api.spotify.com/v1/tracks/4OLT65TRsOx3...,https://api.spotify.com/v1/audio-analysis/4OLT...,181065,1
47,End of Beginning,Djo,spotify:track:3qhlB30KknSejmIvZZLjOD,0.689,0.454,2,-7.643,1,0.0584,0.03510,...,0.0707,0.912,159.982,audio_features,3qhlB30KknSejmIvZZLjOD,spotify:track:3qhlB30KknSejmIvZZLjOD,https://api.spotify.com/v1/tracks/3qhlB30KknSe...,https://api.spotify.com/v1/audio-analysis/3qhl...,159246,4
48,FE!N (feat. Playboi Carti),"Travis Scott, Playboi Carti",spotify:track:42VsgItocQwOQC3XWZ8JNA,0.569,0.882,3,-2.777,0,0.0600,0.03160,...,0.1420,0.201,148.038,audio_features,42VsgItocQwOQC3XWZ8JNA,spotify:track:42VsgItocQwOQC3XWZ8JNA,https://api.spotify.com/v1/tracks/42VsgItocQwO...,https://api.spotify.com/v1/audio-analysis/42Vs...,191701,4
49,In the End,Linkin Park,spotify:track:60a0Rd6pjrkxjPbaKzXjfq,0.556,0.864,3,-5.870,0,0.0584,0.00958,...,0.2090,0.400,105.143,audio_features,60a0Rd6pjrkxjPbaKzXjfq,spotify:track:60a0Rd6pjrkxjPbaKzXjfq,https://api.spotify.com/v1/tracks/60a0Rd6pjrkx...,https://api.spotify.com/v1/audio-analysis/60a0...,216880,4


In [60]:
# Save to CSV
df_global.to_csv('df_global.csv', index=False)